In [2]:
# Notebook cell: SSO inclination & RAAN from date/LTAN/altitude
# Edit DATE_ISO, ALT_KM_LIST, and LTAN_LIST, then run.

from __future__ import annotations
import math
from datetime import datetime, timezone
import pandas as pd

# ------------------- CONFIG (edit these) -------------------
DATE_ISO     = "2027-03-01T12:00:00Z"
ALT_KM_LIST  = [500, 600]
LTAN_LIST    = [6, 9, 12, 15]      # hours
SAVE_CSV     = True
OUT_CSV_PATH = f"sso_design_grid_{DATE_ISO.replace(':','').replace('-','').replace('T','_').replace('Z','Z')}.csv"
# -----------------------------------------------------------

# ---- Earth constants (km, s) ----
MU_EARTH     = 398600.4418        # km^3 / s^2
R_EARTH_EQ   = 6378.137           # km (equatorial)
J2           = 1.08262668e-3
TROPICAL_YR  = 365.2422 * 86400.0 # seconds

def parse_iso_datetime(s: str) -> datetime:
    s = s.strip().replace("Z", "+00:00")
    return datetime.fromisoformat(s).astimezone(timezone.utc)

def wrap_deg(x: float) -> float:
    return (x % 360.0 + 360.0) % 360.0

# ---- Solar RA (degrees) on UTC date/time; dependency-free approximation ----
def sun_ra_deg(dt: datetime) -> float:
    # Julian centuries from J2000.0
    y, m = dt.year, dt.month
    d = dt.day + (dt.hour + (dt.minute + dt.second/60.0)/60.0)/24.0
    if m <= 2:
        y -= 1; m += 12
    A = math.floor(y/100)
    B = 2 - A + math.floor(A/4)
    JD = math.floor(365.25*(y + 4716)) + math.floor(30.6001*(m + 1)) + d + B - 1524.5
    T = (JD - 2451545.0)/36525.0

    # Mean longitude and anomaly (deg)
    L0 = 280.46646 + 36000.76983*T + 0.0003032*T*T
    M  = 357.52911 + 35999.05029*T - 0.0001537*T*T

    # Equation of center (deg)
    Mr = math.radians(M)
    C = (1.914602 - 0.004817*T - 0.000014*T*T)*math.sin(Mr) \
        + (0.019993 - 0.000101*T)*math.sin(2*Mr) \
        + 0.000289*math.sin(3*Mr)

    # Apparent ecliptic longitude (deg)
    true_long  = L0 + C
    Omega      = 125.04 - 1934.136*T
    lambda_app = true_long - 0.00569 - 0.00478*math.sin(math.radians(Omega))

    # Obliquity (deg)
    eps0 = 23.439291 - 0.0130042*T - 1.64e-7*T*T + 5.04e-7*T*T*T
    eps  = eps0 + 0.00256*math.cos(math.radians(Omega))

    # RA
    lam_r = math.radians(lambda_app)
    eps_r = math.radians(eps)
    yv = math.cos(eps_r)*math.sin(lam_r)
    xv = math.cos(lam_r)
    ra = math.degrees(math.atan2(yv, xv))
    return wrap_deg(ra)

def sso_inclination_deg(alt_km: float) -> float:
    """Sun-synchronous inclination from J2 nodal precession (circular orbit)."""
    a   = R_EARTH_EQ + alt_km
    n   = math.sqrt(MU_EARTH / a**3)               # rad/s
    dOm = 2*math.pi / TROPICAL_YR                  # target +rad/s (tracks Sun)
    denom = 1.5 * J2 * n * (R_EARTH_EQ/a)**2
    cos_i = -dOm / denom
    cos_i = max(-1.0, min(1.0, cos_i))
    return math.degrees(math.acos(cos_i))          # retrograde (>90°)

def raan_from_ltan(dt: datetime, ltan_hours: float) -> float:
    """RAAN ≈ RA_sun(date) + 15°*(LTAN - 12)."""
    ra_sun = sun_ra_deg(dt)
    return wrap_deg(ra_sun + 15.0*(ltan_hours - 12.0))

def orbital_period_s(a_km: float) -> float:
    return 2*math.pi * math.sqrt((a_km**3)/MU_EARTH)

# ---- Compute grid ----
dt = parse_iso_datetime(DATE_ISO)
rows = []
for alt in ALT_KM_LIST:
    a_km  = R_EARTH_EQ + float(alt)
    inc   = sso_inclination_deg(float(alt))
    P_s   = orbital_period_s(a_km)
    for ltan in LTAN_LIST:
        raan = raan_from_ltan(dt, float(ltan))
        rows.append({
            "date_utc": dt.isoformat().replace("+00:00","Z"),
            "alt_km": int(alt),
            "a_km": a_km,
            "ltan_h": float(ltan),
            "incl_deg": inc,
            "raan_deg": raan,
            "period_s": P_s,
            "period_min": P_s/60.0,
        })

df = pd.DataFrame(rows).sort_values(["alt_km","ltan_h"]).reset_index(drop=True)

# Round for display
df_disp = df.copy()
for c, nd in [("a_km",3),("incl_deg",4),("raan_deg",3),("period_s",2),("period_min",3)]:
    df_disp[c] = df_disp[c].round(nd)

display(df_disp)

if SAVE_CSV:
    df_disp.to_csv(OUT_CSV_PATH, index=False)
    print("Saved:", OUT_CSV_PATH)


,date_utc,alt_km,a_km,ltan_h,incl_deg,raan_deg,period_s,period_min
0,2027-03-01T12:00:00Z,500,6878.137,6.0,97.4018,252.16,5676.98,94.616
1,2027-03-01T12:00:00Z,500,6878.137,9.0,97.4018,297.16,5676.98,94.616
2,2027-03-01T12:00:00Z,500,6878.137,12.0,97.4018,342.16,5676.98,94.616
3,2027-03-01T12:00:00Z,500,6878.137,15.0,97.4018,27.16,5676.98,94.616
4,2027-03-01T12:00:00Z,600,6978.137,6.0,97.7877,252.16,5801.23,96.687
5,2027-03-01T12:00:00Z,600,6978.137,9.0,97.7877,297.16,5801.23,96.687
6,2027-03-01T12:00:00Z,600,6978.137,12.0,97.7877,342.16,5801.23,96.687
7,2027-03-01T12:00:00Z,600,6978.137,15.0,97.7877,27.16,5801.23,96.687


Saved: sso_design_grid_20270301_120000Z.csv


In [ ]:
# Notebook cell: Compute & save SSO design grid + optional charts
# - Inputs: UTC date, list of altitudes (km), list of LTANs (hours)
# - Outputs:
#     * CSV with columns: date_utc, alt_km, a_km, ltan_h, incl_deg, raan_deg, period_s, period_min
#     * (optional) PNG charts: inclination vs altitude, period vs altitude, RAAN vs LTAN

from __future__ import annotations
import math
from datetime import datetime, timezone
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# ------------------- CONFIG (edit these) -------------------
DATE_ISO     = "2027-03-01T12:00:00Z"
ALT_KM_LIST  = [500, 600]
LTAN_LIST    = [6, 9, 12, 15]        # hours
OUTDIR       = Path(".")     # "." to save in current folder
SAVE_CSV     = True                  # write CSVs
MAKE_PLOTS   = True                  # save PNG charts
# -----------------------------------------------------------

# ---- Earth constants (km, s) ----
MU_EARTH     = 398600.4418        # km^3 / s^2
R_EARTH_EQ   = 6378.137           # km (equatorial)
J2           = 1.08262668e-3
TROPICAL_YR  = 365.2422 * 86400.0 # seconds

def parse_iso_datetime(s: str) -> datetime:
    s = s.strip().replace("Z", "+00:00")
    return datetime.fromisoformat(s).astimezone(timezone.utc)

def wrap_deg(x: float) -> float:
    return (x % 360.0 + 360.0) % 360.0

# ---- Solar RA (degrees) on UTC date/time; dependency-free approximation ----
def sun_ra_deg(dt: datetime) -> float:
    # Julian centuries from J2000.0
    y, m = dt.year, dt.month
    d = dt.day + (dt.hour + (dt.minute + dt.second/60.0)/60.0)/24.0
    if m <= 2:
        y -= 1; m += 12
    A = math.floor(y/100)
    B = 2 - A + math.floor(A/4)
    JD = math.floor(365.25*(y + 4716)) + math.floor(30.6001*(m + 1)) + d + B - 1524.5
    T = (JD - 2451545.0)/36525.0

    # Mean longitude and anomaly (deg)
    L0 = 280.46646 + 36000.76983*T + 0.0003032*T*T
    M  = 357.52911 + 35999.05029*T - 0.0001537*T*T

    # Equation of center (deg)
    Mr = math.radians(M)
    C = (1.914602 - 0.004817*T - 0.000014*T*T)*math.sin(Mr) \
        + (0.019993 - 0.000101*T)*math.sin(2*Mr) \
        + 0.000289*math.sin(3*Mr)

    # Apparent ecliptic longitude (deg)
    true_long  = L0 + C
    Omega      = 125.04 - 1934.136*T
    lambda_app = true_long - 0.00569 - 0.00478*math.sin(math.radians(Omega))

    # Obliquity (deg)
    eps0 = 23.439291 - 0.0130042*T - 1.64e-7*T*T + 5.04e-7*T*T*T
    eps  = eps0 + 0.00256*math.cos(math.radians(Omega))

    # RA
    lam_r = math.radians(lambda_app)
    eps_r = math.radians(eps)
    yv = math.cos(eps_r)*math.sin(lam_r)
    xv = math.cos(lam_r)
    ra = math.degrees(math.atan2(yv, xv))
    return wrap_deg(ra)

def sso_inclination_deg(alt_km: float) -> float:
    """Sun-synchronous inclination from J2 nodal precession (circular orbit)."""
    a   = R_EARTH_EQ + alt_km
    n   = math.sqrt(MU_EARTH / a**3)               # rad/s
    dOm = 2*math.pi / TROPICAL_YR                  # target +rad/s (tracks Sun)
    denom = 1.5 * J2 * n * (R_EARTH_EQ/a)**2
    cos_i = -dOm / denom
    cos_i = max(-1.0, min(1.0, cos_i))
    return math.degrees(math.acos(cos_i))          # retrograde (>90°)

def raan_from_ltan(dt: datetime, ltan_hours: float) -> float:
    """RAAN ≈ RA_sun(date) + 15°*(LTAN - 12)."""
    ra_sun = sun_ra_deg(dt)
    return wrap_deg(ra_sun + 15.0*(ltan_hours - 12.0))

def orbital_period_s(a_km: float) -> float:
    return 2*math.pi * math.sqrt((a_km**3)/MU_EARTH)

# ---- Compute grid ----
dt = parse_iso_datetime(DATE_ISO)
rows = []
for alt in ALT_KM_LIST:
    a_km  = R_EARTH_EQ + float(alt)
    inc   = sso_inclination_deg(float(alt))
    P_s   = orbital_period_s(a_km)
    for ltan in LTAN_LIST:
        raan = raan_from_ltan(dt, float(ltan))
        rows.append({
            "date_utc": dt.isoformat().replace("+00:00","Z"),
            "alt_km": int(alt),
            "a_km": a_km,
            "ltan_h": float(ltan),
            "incl_deg": inc,
            "raan_deg": raan,
            "period_s": P_s,
            "period_min": P_s/60.0,
        })

df = pd.DataFrame(rows).sort_values(["alt_km","ltan_h"]).reset_index(drop=True)

# Round for display
df_disp = df.copy()
for c, nd in [("a_km",3),("incl_deg",4),("raan_deg",3),("period_s",2),("period_min",3)]:
    df_disp[c] = df_disp[c].round(nd)

# ---- Save CSVs ----
OUTDIR.mkdir(parents=True, exist_ok=True)
timestamped = OUTDIR / f"sso_design_grid_{dt.strftime('%Y%m%dT%H%M%SZ')}.csv"
simple      = OUTDIR / "sso_design_grid.csv"
if SAVE_CSV:
    df_disp.to_csv(timestamped, index=False)
    df_disp.to_csv(simple, index=False)

# ---- Show table in notebook ----
try:
    display(df_disp)
except NameError:
    print(df_disp.to_string(index=False))

# ---- Optional charts ----
if MAKE_PLOTS:
    # 1) Inclination vs Altitude
    inc_by_alt = df_disp.groupby("alt_km", as_index=False)["incl_deg"].first()
    fig1, ax1 = plt.subplots(figsize=(7,5))
    bars1 = ax1.bar(inc_by_alt["alt_km"].astype(str).values, inc_by_alt["incl_deg"].values)
    try:
        ax1.bar_label(bars1, padding=3, fmt="%.3f")
    except Exception:
        pass
    ax1.set_title("Sun-synchronous Inclination vs Altitude")
    ax1.set_xlabel("Altitude (km)")
    ax1.set_ylabel("Inclination (deg)")
    ax1.grid(True, axis="y")
    fig1.tight_layout()
    fig1.savefig(OUTDIR / "sso_inclination_vs_altitude.png", bbox_inches="tight")
    plt.close(fig1)

    # 2) Orbital Period vs Altitude
    per_by_alt = df_disp.groupby("alt_km", as_index=False)["period_min"].first()
    fig2, ax2 = plt.subplots(figsize=(7,5))
    bars2 = ax2.bar(per_by_alt["alt_km"].astype(str).values, per_by_alt["period_min"].values)
    try:
        ax2.bar_label(bars2, padding=3, fmt="%.2f")
    except Exception:
        pass
    ax2.set_title("Orbital Period vs Altitude")
    ax2.set_xlabel("Altitude (km)")
    ax2.set_ylabel("Period (minutes)")
    ax2.grid(True, axis="y")
    fig2.tight_layout()
    fig2.savefig(OUTDIR / "sso_period_vs_altitude.png", bbox_inches="tight")
    plt.close(fig2)

    # 3) RAAN vs LTAN
    raan_by_ltan = df_disp.groupby("ltan_h", as_index=False)["raan_deg"].mean().sort_values("ltan_h")
    fig3, ax3 = plt.subplots(figsize=(8,5))
    ax3.plot(raan_by_ltan["ltan_h"].values, raan_by_ltan["raan_deg"].values, marker="o")
    ax3.set_title("RAAN vs LTAN (at specified UTC date)")
    ax3.set_xlabel("LTAN (hours)")
    ax3.set_ylabel("RAAN (deg)")
    ax3.set_xticks(sorted(df_disp["ltan_h"].unique()))
    ax3.grid(True)
    fig3.tight_layout()
    fig3.savefig(OUTDIR / "sso_raan_vs_ltan.png", bbox_inches="tight")
    plt.close(fig3)

# Handy printouts
print("Saved CSVs to:", timestamped, "and", simple)
if MAKE_PLOTS:
    print("Saved PNGs to:", OUTDIR / "sso_inclination_vs_altitude.png",
          OUTDIR / "sso_period_vs_altitude.png",
          OUTDIR / "sso_raan_vs_ltan.png")


,date_utc,alt_km,a_km,ltan_h,incl_deg,raan_deg,period_s,period_min
0,2027-03-01T12:00:00Z,500,6878.137,6.0,97.4018,252.16,5676.98,94.616
1,2027-03-01T12:00:00Z,500,6878.137,9.0,97.4018,297.16,5676.98,94.616
2,2027-03-01T12:00:00Z,500,6878.137,12.0,97.4018,342.16,5676.98,94.616
3,2027-03-01T12:00:00Z,500,6878.137,15.0,97.4018,27.16,5676.98,94.616
4,2027-03-01T12:00:00Z,600,6978.137,6.0,97.7877,252.16,5801.23,96.687
5,2027-03-01T12:00:00Z,600,6978.137,9.0,97.7877,297.16,5801.23,96.687
6,2027-03-01T12:00:00Z,600,6978.137,12.0,97.7877,342.16,5801.23,96.687
7,2027-03-01T12:00:00Z,600,6978.137,15.0,97.7877,27.16,5801.23,96.687


Saved CSVs to: \orbit_analysis\sso_design_grid_20270301T120000Z.csv and \orbit_analysis\sso_design_grid.csv
Saved PNGs to: \orbit_analysis\sso_inclination_vs_altitude.png \orbit_analysis\sso_period_vs_altitude.png \orbit_analysis\sso_raan_vs_ltan.png
